# -

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd drive/project_data

/content/drive/project_data


In [0]:
cd Yolo_type_data/

/content/drive/project_data/Yolo_type_data


In [0]:
pwd

'/content/drive/project_data/Yolo_type_data'

In [0]:
import keras
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras.layers.normalization import BatchNormalization
from keras import optimizers
import numpy as np
import tensorflow as tf
from keras.optimizers import SGD

Using TensorFlow backend.


In [0]:
X = np.load('X.npy')
Y = np.load('ninja.npy')
print(X.shape)
print(Y.shape)

(21503, 300, 300, 3)
(21503, 49, 25)


# Model

In [0]:
def add_vgg(model, shape=(300,300,3)):
    
    # Block 1
    model.add(Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1',input_shape=shape,
                      trainable=False))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2',
                      trainable=False))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
    model.add(BatchNormalization())

    # Block 2
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1',
                      trainable=False))
    model.add(Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2',
                      trainable=False))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
    model.add(BatchNormalization())

    # Block 3
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1',
                      trainable=False))
    model.add(Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2',
                      trainable=False))
    model.add(Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3',
                      trainable=False))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
    model.add(BatchNormalization())

    # Block 4
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1',
                      trainable=False))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2',
                      trainable=False))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3',
                      trainable=False))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
    model.add(BatchNormalization())

    # Block 5
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1',
                      trainable=False))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2',
                      trainable=False))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3',
                      trainable=False))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
    model.add(BatchNormalization())

    return model

#VGG structure model





In [0]:
def add_extra_layers(model):
    #extra Block
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='extra_block_conv1'))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='extra_block_conv2'))
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='extra_block_conv3'))
    
    
    model.add(BatchNormalization())
    
    return model

In [0]:
def add_final_layer(model):
    
    #final extra Block
    model.add(Conv2D(512, (3, 3),
                      activation='relu',
                      name='final_block1_conv1'))

    model.add(Flatten()) # 25088 output
    model.add(Dropout(0.2))
    model.add(Dense(4000,activation='relu'))
    model.add(Dense(1225,activation='relu'))
    model.add(Reshape((49, 25))) # 49*25 = 1225
    model.add(Dense(25,activation='relu',input_shape=(49,25)))
    return model

In [0]:
def create_model(inp_shape=(300,300,3)):
    #Declaring Sequential model
    model = Sequential()
    # Loading VGG model structure
    model = add_vgg(model)
    
    #model.load_weights('classifier_weights.h5',by_name=True)
    model.load_weights('Ultra_Gandu_Ninja_Model_epoch5.h5',by_name=True)
    model = add_extra_layers(model)
    model = add_final_layer(model)
    # Loading trained VGG weights to the named layers
    
    
    
    return model
    

# Create Model

In [0]:
model = create_model()


# Losses and Part-Wise Accuracies

In [0]:
# This is the loss function that we are optimizzing on

def C_Loss(Y1, Y2):
  
  Y_true = tf.cast(Y1, tf.float32)
  Y_pred = tf.cast(Y2, tf.float32)
  
  loss1 = tf.log(tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,0:1],Y_true[:,:,0:1]))) + 1)
  loss2 = tf.log(tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,1:2],Y_true[:,:,1:2]))) + 1)
  loss3 = tf.log(tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,2:3],Y_true[:,:,2:3]))) + 1)
  loss4 = tf.log(tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,3:4],Y_true[:,:,3:4]))) + 1)
  
  loss5 = tf.log(tf.square(tf.subtract(Y_true[:,:,4:5],Y_pred[:,:,4:5])) + 1)

  loss6 = tf.log(tf.square(tf.subtract(Y_true[:,:,5:25], Y_pred[:,:,5:25])) + 1)

  return (5 * (loss1+loss2+loss3+loss4) + 0.5 * (loss5) + 0.5 * loss6)

In [0]:

def C_LossABC(Y1, Y2):
  
  Y_true = tf.cast(Y1, tf.float32) * 10
  Y_pred = tf.cast(Y2, tf.float32) * 10
  
  loss1 = tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,0:1],Y_true[:,:,0:1])))
  loss2 = tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,1:2],Y_true[:,:,1:2])))
  loss3 = tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,2:3],Y_true[:,:,2:3])))
  loss4 = tf.multiply(Y_true[:,:,4:5],tf.square(tf.subtract(Y_pred[:,:,3:4],Y_true[:,:,3:4])))
  
  loss5 = tf.log(tf.square(tf.subtract(Y_true[:,:,4:5],Y_pred[:,:,4:5])) + 1)

  loss6 = tf.log(tf.square(tf.subtract(Y_true[:,:,5:25], Y_pred[:,:,5:25])) + 1)

  return (5 * (loss1+loss2+loss3+loss4) + 0.5 * (loss5) + 0.5 * loss6)

In [0]:
# Combined accurecy for center(x, y), height(h), weidth(w)
from keras import backend as K
def bbox_loss(Y1, Y2):

  Y_true = tf.cast(Y1[:,:,0:4], tf.float32) * 300
  Y_pred = tf.cast(Y2[:,:,0:4], tf.float32) * 300
  
  return K.mean(K.square(Y_pred - Y_true), axis=-1)
  #return tf.divide(tf.abs(tf.subtract(Y_true, Y_pred)), Y_true)
  
  

In [0]:
# Class Prediction loss (categorical_cross_entropy)
from keras import backend as K
def class_pred_loss(Y1, Y2):
  Y_true = tf.cast(Y1[:,:,5:], tf.float32)
  Y_pred = tf.cast(Y2[:,:,5:], tf.float32)
  
  return K.categorical_crossentropy(Y_true, Y_pred)

# Training Part 1

In [0]:
sgd = SGD(lr=0.001)
model.compile(loss=C_LossABC,
              optimizer=sgd,
              metrics=[bbox_loss, class_pred_loss])

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

Train on 16000 samples, validate on 5503 samples
Epoch 1/5
13475/16000 [========================>.....] - ETA: 1:54 - loss: 353.9790 - bbox_loss: 1582.4964 - class_pred_loss: 5.7971e-09

16000/16000 [==============================] - 860s 54ms/step - loss: 353.1560 - bbox_loss: 1578.7932 - class_pred_loss: 5.7739e-09 - val_loss: 351.9188 - val_bbox_loss: 1573.2268 - val_class_pred_loss: 5.5974e-09
Epoch 2/5
 6175/16000 [==========>...................] - ETA: 7:22 - loss: 352.7229 - bbox_loss: 1576.8441 - class_pred_loss: 5.7766e-09

16000/16000 [==============================] - 857s 54ms/step - loss: 353.1560 - bbox_loss: 1578.7932 - class_pred_loss: 5.7739e-09 - val_loss: 351.9188 - val_bbox_loss: 1573.2268 - val_class_pred_loss: 5.5974e-09
Epoch 3/5
 3550/16000 [=====>........................] - ETA: 9:21 - loss: 362.2685 - bbox_loss: 1619.7985 - class_pred_loss: 5.9896e-09

16000/16000 [==============================] - 857s 54ms/step - loss: 353.1560 - bbox_loss: 1578.7932 - class_pred_loss: 5.7739e-09 - val_loss: 351.9188 - val_bbox_loss: 1573.2268 - val_class_pred_loss: 5.5974e-09
Epoch 4/5
 2625/16000 [===>..........................] - ETA: 10:02 - loss: 348.5150 - bbox_loss: 1557.9091 - class_pred_loss: 5.7350e-09

16000/16000 [==============================] - 857s 54ms/step - loss: 353.1560 - bbox_loss: 1578.7932 - class_pred_loss: 5.7739e-09 - val_loss: 351.9188 - val_bbox_loss: 1573.2268 - val_class_pred_loss: 5.5974e-09
Epoch 5/5
 2275/16000 [===>..........................] - ETA: 10:18 - loss: 352.3395 - bbox_loss: 1575.1192 - class_pred_loss: 5.7030e-09

16000/16000 [==============================] - 857s 54ms/step - loss: 353.1560 - bbox_loss: 1578.7932 - class_pred_loss: 5.7739e-09 - val_loss: 351.9188 - val_bbox_loss: 1573.2268 - val_class_pred_loss: 5.5974e-09


In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch4.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

Train on 16000 samples, validate on 5503 samples
Epoch 1/5


ResourceExhaustedError: ignored

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch10.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch15.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch20.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch25.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch30.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=25, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch35.h5')

# Training Part 2

In [0]:
#model.load_weights('Ultra_Gandu_Ninja_Weights_epoch35.h5',by_name=True)

In [0]:
sgd = SGD(lr=0.0001)
model.compile(loss=C_Loss,
              optimizer=sgd,
              metrics=[bbox_loss, class_pred_loss])

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=64, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch40.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=64, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch45.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=64, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch50.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=64, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch55.h5')

In [0]:
model.fit(X[0:16000], Y[0:16000], batch_size=64, epochs=5, verbose=1,validation_data=(X[16000:],Y[16000:]))

In [0]:
model.save('Ultra_Gandu_Ninja_Model_epoch60.h5')

# Test Preds

In [0]:
pred = model.predict(X[3:4])

In [0]:
pred[0,0]

In [0]:
Y[3,0]

In [0]:
sess = tf.Session()
pred = model.predict(X[0:1])
true = np.zeros((1,49,25), dtype='float64')
true[0] = Y[0]

error = C_Loss(true,pred)
print(sess.run(tf.reduce_mean(error)))

In [0]:
Y.shape

# Create lable

In [0]:
Y = np.load('49*6lable.npy')
k = np.zeros((21503, 49, 25))
for i in range(21503):
  for j in range(49):
    
    # x, y, h**0.5, w**0.5, confidence,
    k[i,j, 0] = Y[i,j,1]
    k[i,j, 1] = Y[i,j,2]
    k[i,j, 2] = Y[i,j,3] ** 0.5
    k[i,j, 3] = Y[i,j,4] ** 0.5
    k[i,j, 4] = Y[i,j,5]
    
    index = int(4 + Y[i,j,0])
    k[i][j][index] = 1

np.save("ninja.npy", k)

In [0]:
ls

In [0]:
Y[0,0]

In [0]:
Y[0][0]

In [0]:
model.predict(X[2:3])

In [0]:
Y[2]